In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
import statistics as stats
from os import listdir
from skimage import io, filters, color,exposure,feature,measure,segmentation
from scipy import ndimage
import cv2 
import imutils
import glob
import sys
sys.path.append('../scripts/')
from nuclei_segmentation import nuclei_segmenter

In [ ]:
seg = nuclei_segmenter(area_threshold=250,median_filter_param=0,cropped_flag=0)

In [ ]:
Herlev_df_ext = pd.DataFrame()
folders = listdir('../data/Herlev Database Pictures/')[1:]
for folder in folders:
    for blob in glob.glob('../data/Herlev Database Pictures/{}/*.BMP'.format(folder)):
        df = seg.segment_nuclei(blob)
        df['class'] = folder
        if folder.startswith('normal'):
            df['normal']=1
        else:
            df['normal']=0
        Herlev_df_ext = Herlev_df_ext.append(df)

In [ ]:
from sklearn.model_selection import train_test_split
X = Herlev_df_ext.drop(['class','normal','ID'],axis=1)
y = Herlev_df['normal']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.30,stratify=y)


In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.datasets import make_classification
X, y = make_classification(n_samples=1000, n_features=4,n_informative=2, n_redundant=0,random_state=0, shuffle=False)
clf = RandomForestClassifier(max_depth=2, random_state=0)
clf.fit(X_train, y_train)


In [ ]:
from sklearn.metrics import classification_report,confusion_matrix

In [ ]:
preds = clf.predict(X_test)
print(classification_report(y_test,preds))

In [ ]:
important_features_dict = {}
for x,i in enumerate(clf.feature_importances_):
    important_features_dict[list(X_train.keys())[x]]=i

sorted_features = sorted(important_features_dict.items(), key=lambda kv: kv[1], reverse=True)
from collections import OrderedDict
OrderedDict(sorted_features)

In [ ]:
#return image with suspicious nuclei flagged 